In [82]:
import requests
from requests.exceptions import HTTPError
from bs4 import BeautifulSoup as soup
import re
import urllib.parse
from urllib.request import Request, urlopen
from urllib.parse import urlparse
import random
import json

In [83]:
def googleSearch(query):
    g_clean = [ ] 
    url = 'https://www.google.com/search?client=ubuntu&channel=fs&q={}&ie=utf-8&oe=utf-8'.format(query) 
    try:
            html = requests.get(url)
            if html.status_code==200:
                page_soup = soup(html.text, 'lxml')
                a = page_soup.find_all('a') 
                for i in a:
                    k = i.get('href')
                    try:
                        m = re.search("(?P<url>https?://[^\s]+)", k)
                        n = m.group(0)
                        rul = n.split('&')[0]
                        domain = urlparse(rul)
                        if(re.search('google.com', domain.netloc)):
                            continue
                        else:
                            g_clean.append(rul)
                    except:
                        continue
    except Exception as ex:
            print(str(ex))
    finally:
            return g_clean

In [84]:
googleSearch('right arm pain')

['https://www.medicalnewstoday.com/articles/327172',
 'https://www.medicalnewstoday.com/articles/321528',
 'https://www.medicalnewstoday.com/articles/151444',
 'https://www.medicalnewstoday.com/articles/326895',
 'https://www.medicalnewstoday.com/articles/321902',
 'https://www.mayoclinic.org/symptoms/arm-pain/basics/when-to-see-doctor/sym-20050870',
 'https://www.webmd.com/a-to-z-guides/why-does-my-arm-hurt',
 'https://www.omron-healthcare.com/eu/health-and-lifestyle/pain-management/body-aches-and-pains/right-arm-pain-symptoms-causes-and-treatment.html',
 'https://www.healthline.com/health/arm-pain',
 'https://www.healthline.com/health/arm-pain%23causes',
 'https://www.healthline.com/health/arm-pain%23diagnosis',
 'https://www.healthline.com/health/arm-pain%23home-remedies',
 'https://www.healthline.com/health/arm-pain%23prevention',
 'https://www.honorhealth.com/medical-services/cardiac-care/heart-health/heart-attack-symptoms-and-warning-signs',
 'https://myheartsisters.org/2013/07/1

In [85]:
def randomAgent():
    
    randomAgents = [
        'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5',
        'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; ko; rv:1.9.1b2) Gecko/20081201 Firefox/3.1b2'        ,
        'Mozilla/5.0 (X11; U; SunOS sun4u; en-US; rv:1.9b5) Gecko/2008032620 Firefox/3.0b5',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0b8pre) Gecko/20101114 Firefox/4.0b8pre',
        'Mozilla/5.0 (X11; Linux x86_64; rv:2.0b9pre) Gecko/20110111 Firefox/4.0b9pre'
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b9pre) Gecko/20101228 Firefox/4.0b9pre'
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.2a1pre) Gecko/20110324 Firefox/4.2a1pre'
        'Mozilla/5.0 (X11; U; Linux amd64; rv:5.0) Gecko/20100101 Firefox/5.0 (Debian)'
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0a2) Gecko/20110613 Firefox/6.0a2'
        'Mozilla/5.0 (X11; Linux i686 on x86_64; rv:12.0) Gecko/20100101 Firefox/12.0'
                   ]
    
    return random.choice(randomAgents)

In [86]:
def scrape(url):
        
        req = Request(url, headers={'User-Agent' : randomAgent()})
        
        webpage = urlopen(req).read()
        return webpage

In [87]:
page = scrape('https://www.webmd.com/pain-management/lower-leg-pain-causes-and-treatments')

In [88]:
def scrapeByTag(webpage, tag):
    
        page_soup = soup(webpage,'html.parser')
        taglist = page_soup.findAll(tag)
        
        return taglist

In [89]:
def scrapeByClassName(webpage, tag, classname):

        page_soup = soup(webpage,'html.parser')
        classlist = page_soup.findAll(tag, class_=classname)
        
        return classlist

In [90]:
def formatText(text, i): 
    
        text = str(text)
    
        cleaned = re.sub(r"<{}>|<{}>", "", text).format(i)
        
        return cleaned

In [91]:
def extract(webpage):
    
    extractions = {}
    
    tagTypes = ['title', 'h','h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'a', 'meta']
    
    for i in tagTypes:
        
        
        e = scrapeByTag(webpage, i)
        formatted = []
        
        for item in e:
            
            formatted.append(str(item))
            
        extractions[i] = formatted
    
    return extractions

In [92]:
def meta(webpage):
    
    metaTags = {}
    
    types = ['meta']
    
    for i in tagTypes:
        
        
        e = scrapeByTag(webpage, i)
        
        for item in e:
            item = formatText(item, i)
        
        extractions[i] = e
    
    return metaTags

In [93]:
def classnameSearch(taglist, keyword):
    
    retrieved = []
    
    for i in taglist:
        
        if keyword in str(i):    
            retrieved.append(i)
            
        
    return retrieved

In [94]:
def toJSON(title, obj, location):
    with open(location + '/' + title + '.json', 'w', encoding='utf-8') as f:
        json.dump(obj, f, ensure_ascii=False, indent=4)
    return

In [108]:
def createResources(query, folder):
    
    links = googleSearch(query)
    count = 0
    
    for url in links: 
        
        print('processing... rank {} : '.format(count) + url)
        
        
        try:

            page = scrape(url)
            extraction = extract(page)
            extraction['site address'] = url
            toJSON('Site_' + str(count), extraction, folder)

            count += 1
            
        except HTTPError:
            print('\n' + 'HTTP Error Exception Handeling --> skipping')
            count += 1
            continue
    
    print ('\nprocessing complete')
    return

In [110]:
createResources('arm pain', 'Extractions/Arm Pain')

processing... rank 0 : https://www.mayoclinic.org/symptoms/arm-pain/basics/causes/sym-20050870
processing... rank 1 : https://www.mayoclinic.org/symptoms/arm-pain/basics/when-to-see-doctor/sym-20050870
processing... rank 2 : https://www.webmd.com/a-to-z-guides/why-does-my-arm-hurt
processing... rank 3 : https://www.healthline.com/health/arm-pain
processing... rank 4 : https://www.healthline.com/health/arm-pain%23causes
processing... rank 5 : https://www.healthline.com/health/arm-pain%23diagnosis
processing... rank 6 : https://www.healthline.com/health/arm-pain%23home-remedies
processing... rank 7 : https://www.healthline.com/health/arm-pain%23prevention
processing... rank 8 : https://www.medicinenet.com/arm_pain/symptoms.htm
processing... rank 9 : https://www.medicalnewstoday.com/articles/327172
processing... rank 10 : https://www.nhs.uk/conditions/elbow-and-arm-pain/
processing... rank 11 : https://www.healthlinkbc.ca/health-topics/armlp
processing... rank 12 : https://www.omron-healt